In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/for-h20-training/Xt.csv
/kaggle/input/for-h20-training/X.csv
/kaggle/input/bri-data-hackathon-pa/sample_submission.csv
/kaggle/input/bri-data-hackathon-pa/data_description.csv
/kaggle/input/bri-data-hackathon-pa/train.csv
/kaggle/input/bri-data-hackathon-pa/test.csv


In [2]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

3.32.0.4
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpspfjg9g6
  JVM stdout: /tmp/tmpspfjg9g6/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpspfjg9g6/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_unknownUser_8inmt7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
df_train = pd.read_csv('../input/bri-data-hackathon-pa/train.csv')
df_test = pd.read_csv('../input/bri-data-hackathon-pa/test.csv')
sample_submission = pd.read_csv('../input/bri-data-hackathon-pa/sample_submission.csv')

In [4]:
%%time
train = h2o.import_file("../input/for-h20-training/X.csv")
test = h2o.import_file("../input/for-h20-training/Xt.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
CPU times: user 97.9 ms, sys: 14.2 ms, total: 112 ms
Wall time: 3.29 s


In [5]:
x = test.columns
y = 'Best Performance'
train[y] = train[y].asfactor()

In [6]:
nominal = ['age','year_graduated',
           'job_duration_in_current_job_level',
           'job_duration_in_current_person_level',
           'job_duration_in_current_branch',           
           'GPA',
           'job_duration_from_training',
           'branch_rotation',
           'job_rotation',
           'assign_of_otherposition',
           'annual leave',
           'sick_leaves',
           'Last_achievement_%',
           'Achievement_above_100%_during3quartal',]
ordinal = ['job_level',
           'person_level',
           'Employee_type',
           'number_of_dependences',
           'Education_level',
           'gender','marital_status_maried(Y/N)'
          ]

In [7]:
for col in ordinal:
    train[col] = train[col].asfactor()
    test[col] = test[col].asfactor()

In [8]:
aml = H2OAutoML(max_models=1000, seed=47, nfolds=100, max_runtime_secs=5000)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [9]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20210224_155016,0.59833,0.407808,0.217781,0.431995,0.350056,0.12254
StackedEnsemble_BestOfFamily_AutoML_20210224_155016,0.589259,0.408837,0.212972,0.445612,0.350471,0.12283
GBM_grid__1_AutoML_20210224_155016_model_4,0.588903,0.410323,0.200048,0.438347,0.351334,0.123435
GBM_grid__1_AutoML_20210224_155016_model_1,0.58765,0.413638,0.200359,0.436547,0.352402,0.124187
GBM_grid__1_AutoML_20210224_155016_model_2,0.583148,0.410917,0.206306,0.447624,0.351398,0.123481
GBM_2_AutoML_20210224_155016,0.582876,0.412533,0.195917,0.435468,0.352369,0.124164
XGBoost_grid__1_AutoML_20210224_155016_model_6,0.580202,0.413465,0.198254,0.442875,0.352416,0.124197
GBM_5_AutoML_20210224_155016,0.580189,0.411903,0.200936,0.449111,0.351753,0.12373
GBM_1_AutoML_20210224_155016,0.58011,0.412055,0.19944,0.446082,0.351997,0.123902
XGBoost_grid__1_AutoML_20210224_155016_model_5,0.57806,0.413118,0.201165,0.451897,0.352147,0.124007


In [10]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210224_155016

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.09278948419690616
RMSE: 0.30461366383815774
LogLoss: 0.3152600650871519
Null degrees of freedom: 10014
Residual degrees of freedom: 10002
Null deviance: 8326.000145068137
Residual deviance: 6314.6591036956515
AIC: 6340.6591036956515
AUC: 0.9534193168755597
AUCPR: 0.7607510178247401
Gini: 0.9068386337511194

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1882026600971981: 


,,0,1,Error,Rate
0,0,7973.0,580.0,0.0678,(580.0/8553.0)
1,1,258.0,1204.0,0.1765,(258.0/1462.0)
2,Total,8231.0,1784.0,0.0837,(838.0/10015.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.188203,0.741836,224.0
1,max f2,0.164870,0.813170,251.0
2,max f0point5,0.205578,0.721037,207.0
3,max accuracy,0.205578,0.918922,207.0
4,max precision,0.906302,1.000000,0.0
5,max recall,0.112087,1.000000,338.0
6,max specificity,0.906302,1.000000,0.0
7,max absolute_mcc,0.188203,0.697378,224.0
8,max min_per_class_accuracy,0.171802,0.892613,242.0
9,max mean_per_class_accuracy,0.164870,0.897266,251.0



Gains/Lift Table: Avg response rate: 14.60 %, avg score: 15.46 %


...

,,0,1,Error,Rate
0,0,4817.0,4699.0,0.4938,(4699.0/9516.0)
1,1,606.0,1031.0,0.3702,(606.0/1637.0)
2,Total,5423.0,5730.0,0.4757,(5305.0/11153.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.132818,0.279897,282.0
1,max f2,0.105162,0.466853,354.0
2,max f0point5,0.176473,0.240057,188.0
3,max accuracy,0.446851,0.853941,34.0
4,max precision,0.499737,0.564103,21.0
5,max recall,0.080599,1.000000,393.0
6,max specificity,0.798640,0.999895,0.0
7,max absolute_mcc,0.224302,0.109962,135.0
8,max min_per_class_accuracy,0.137702,0.565058,269.0
9,max mean_per_class_accuracy,0.139713,0.568720,264.0



Gains/Lift Table: Avg response rate: 14.68 %, avg score: 14.68 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010042,0.405021,3.406536,3.406536,0.500000,0.488029,0.500000,0.488029,0.034209,0.034209,240.653635,240.653635,0.028324
1,2,0.020084,0.327248,2.129085,2.767811,0.312500,0.359889,0.406250,0.423959,0.021381,0.055589,112.908522,176.781078,0.041613
2,3,0.030037,0.282680,1.779992,2.440504,0.261261,0.302259,0.358209,0.383635,0.017715,0.073305,77.999197,144.050365,0.050711
3,4,0.040079,0.254063,1.399113,2.179574,0.205357,0.267054,0.319911,0.354424,0.014050,0.087355,39.911314,117.957359,0.055409
4,5,0.050031,0.236531,1.779992,2.100087,0.261261,0.244474,0.308244,0.332552,0.017715,0.105070,77.999197,110.008692,0.064507
5,6,0.100063,0.194858,1.465177,1.782632,0.215054,0.211777,0.261649,0.272165,0.073305,0.178375,46.517692,78.263192,0.091784
6,7,0.150004,0.176382,1.308795,1.624875,0.192101,0.184992,0.238494,0.243142,0.065363,0.243739,30.879493,62.487508,0.109859
7,8,0.200036,0.166395,1.123302,1.499426,0.164875,0.171129,0.220081,0.225131,0.056200,0.299939,12.330231,49.942568,0.117089
8,9,0.300009,0.152688,1.063206,1.354063,0.156054,0.159101,0.198745,0.203128,0.106292,0.406231,6.320596,35.406256,0.124495
9,10,0.399982,0.141950,1.044875,1.276783,0.153363,0.147113,0.187402,0.189127,0.104459,0.510690,4.487483,27.678296,0.129753


In [11]:
preds = aml.predict(test)
preds['p1'].as_data_frame().values.flatten().shape

stackedensemble prediction progress: |████████████████████████████████████| 100%


/opt/conda/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'person_level' has levels not trained on: ["2", "4", "6", "8", "10"]
  warnings.warn(w)
/opt/conda/lib/python3.7/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'number_of_dependences' has levels not trained on: ["2", "5", "8"]
  warnings.warn(w)


(6000,)

In [12]:
sample_submission.shape

(6000, 2)

In [13]:
sample_submission['Best Performance'] = preds['p1'].as_data_frame().values
sample_submission.to_csv('h2o_automl_submission.csv', index=False)

In [14]:
sample_submission['Best Performance']

0       0.100155
1       0.107709
2       0.253823
3       0.087281
4       0.145542
          ...   
5995    0.108268
5996    0.088198
5997    0.128565
5998    0.140087
5999    0.104964
Name: Best Performance, Length: 6000, dtype: float64